In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import catboost as cb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold
from sklearn.metrics import roc_auc_score,mean_squared_error,mean_absolute_error
import xgboost as xgb
from datetime import datetime
import gc
from bayes_opt import BayesianOptimization
from kaggle.competitions import nflrush
import math
import tqdm
from scipy.spatial import Delaunay, delaunay_plot_2d, Voronoi, voronoi_plot_2d, ConvexHull
pd.set_option("display.max_columns",1000)
env = nflrush.make_env()

In [2]:
def get_cdf_df(yards_array):
    pdf, edges = np.histogram(yards_array, bins=199,
                 range=(-99,100), density=True)
    cdf = pdf.cumsum().clip(0, 1)
    cdf_df = pd.DataFrame(data=cdf.reshape(-1, 1).T, 
                            columns=['Yards'+str(i) for i in range(-99,100)])
    return cdf_df

def get_score(y_pred,cdf,w,dist_to_end):
    y_pred = int(y_pred)
    if y_pred ==w:
        y_pred_array = cdf.copy()
    elif y_pred - w >0:
        y_pred_array = np.zeros(199)
        y_pred_array[(y_pred-w):] = cdf[:(-(y_pred-w))].copy()
    elif w - y_pred >0:
        y_pred_array = np.ones(199)
        y_pred_array[:(y_pred-w)] = cdf[(w-y_pred):].copy()
    y_pred_array[-1]=1
    y_pred_array[(dist_to_end+99):]=1
    return y_pred_array    

def get_score_pingyi1(y_pred,y_true,cdf,w,dist_to_end):
    y_pred = int(y_pred)
    if y_pred ==w:
        y_pred_array = cdf.copy()
    elif y_pred - w >0:
        y_pred_array = np.zeros(199)
        y_pred_array[(y_pred-w):] = cdf[:(-(y_pred-w))].copy()
    elif w - y_pred >0:
        y_pred_array = np.ones(199)
        y_pred_array[:(y_pred-w)] = cdf[(w-y_pred):].copy()
    y_pred_array[-1]=1
    y_pred_array[(dist_to_end+99):]=1
    y_true_array = np.zeros(199)
    y_true_array[(y_true+99):]=1
    return np.mean((y_pred_array - y_true_array)**2)

def CRPS_pingyi1(y_preds,y_trues,w,cdf,dist_to_ends):
    if len(y_preds) != len(y_trues):
        print('length does not match')
        return None
    n = len(y_preds)
    tmp = []
    for a,b,c in zip(y_preds, y_trues,dist_to_ends):
        tmp.append(get_score_pingyi1(a,b,cdf,w,c))
    return np.mean(tmp)

def crps_score(y_pred, y_true):
    y_pred = np.array(y_pred).astype("int")
    y_pred_cdf = np.zeros([y_pred.shape[0], 199])
    for i, y in enumerate(y_pred):
        y_pred_cdf[i, 99+y] = 1
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, 99+y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred_cdf = np.clip(np.cumsum(y_pred_cdf, axis=1), 0, 1)
    return ((y_true_cdf - y_pred_cdf) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true_cdf.shape[0])

In [3]:
train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv',low_memory=False)

# preprocess and feature engineering 

In [4]:
def transform_time_quarter(str1):
    return int(str1[:2])*60 + int(str1[3:5])
  
def transform_time_all(str1,quarter):
    if quarter<=4:
        return 15*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    if quarter ==5:
        return 10*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
      
def back_direction(orientation):
    if orientation > 180.0:
        return 1
    else:
        return 0
      
def transform_height(te):
    return (int(te.split('-')[0])*12 + int(te.split('-')[1]))*2.54/100

def voronoi_volumes(points, selected_index):
    v = Voronoi(points)
    vol = np.zeros(v.npoints)
      
    for i, reg_num in enumerate(v.point_region):
        if reg_num == v.point_region[selected_index]:
            indices = v.regions[reg_num]
            if -1 in indices: # some regions can be opened
                vol = -999 ## insert missing value when the area is open
            else:
                vol = ConvexHull(v.vertices[indices]).volume      
            break
    return vol

In [5]:
remove_features = ['PlayId','DisplayName','GameClock','TimeHandoff','TimeSnap', 'PlayDirection', 'TeamOnOffense', 
                   'Turf', 'PlayerBirthDate', 'is_run', 'NflIdRusher', 'date_game', 'FieldPosition', 
                   'HomeTeamAbbr', 'VisitorTeamAbbr', 'PlayerHeight', 'own_field']#'GameId','PossessionTeam','JerseyNumber', "PlayerWeight","WindSpeed", "WindDirection"]

In [6]:
def transform_data(df):
    df.loc[df.VisitorTeamAbbr == "ARI",'VisitorTeamAbbr'] = "ARZ"
    df.loc[df.HomeTeamAbbr == "ARI",'HomeTeamAbbr'] = "ARZ"

    df.loc[df.VisitorTeamAbbr == "BAL",'VisitorTeamAbbr'] = "BLT"
    df.loc[df.HomeTeamAbbr == "BAL",'HomeTeamAbbr'] = "BLT"

    df.loc[df.VisitorTeamAbbr == "CLE",'VisitorTeamAbbr'] = "CLV"
    df.loc[df.HomeTeamAbbr == "CLE",'HomeTeamAbbr'] = "CLV"

    df.loc[df.VisitorTeamAbbr == "HOU",'VisitorTeamAbbr'] = "HST"
    df.loc[df.HomeTeamAbbr == "HOU",'HomeTeamAbbr'] = "HST"

    df['is_run'] = df.NflId == df.NflIdRusher

    if 2017 in list(df["Season"].unique()):
        df.loc[df['Season'] == 2017, 'S'] = (df['S'][df['Season'] == 2017] - 2.4355) / 1.2930 * 1.4551 + 2.7570

    # standardization -----------------------------
    df['ToLeft'] = df.PlayDirection == "left"
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['OnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?
    df['YardLine_std'] = 100 - df.YardLine.copy()
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
            'YardLine_std'
             ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
              'YardLine']
    df['X_std'] = df.X.copy()
    df.loc[df.ToLeft, 'X_std'] = 120 - df.loc[df.ToLeft, 'X'] 
    df['Y_std'] = df.Y.copy()
    df.loc[df.ToLeft, 'Y_std'] = 53.3 - df.loc[df.ToLeft, 'Y'] 
    df['Orientation_std'] = df.Orientation.copy()
    df.loc[df.ToLeft, 'Orientation_std'] = np.mod(180 + df.loc[df.ToLeft, 'Orientation_std'], 360)
    df['Dir_std'] = df.Dir.copy()
    df.loc[df.ToLeft, 'Dir_std'] = np.mod(180 + df.loc[df.ToLeft, 'Dir_std'], 360)
    df.loc[df['Season'] == 2017, 'Orientation_std'] = np.mod(90 + df.loc[df['Season'] == 2017, 'Orientation_std'], 360) 
    df.drop(["X", "Y", "Orientation", "YardLine", "Dir", "ToLeft"], axis=1, inplace=True)
    df.rename(columns={'X_std': 'X', 'Y_std': 'Y', 'Orientation_std': 'Orientation', 'Dir_std': 'Dir', "YardLine_std": "YardLine"}, inplace=True)
    # standardization -----------------------------
    
    df['date_game'] = df.GameId.map(lambda x:pd.to_datetime(str(x)[:8]))
    df['age'] = (df.date_game.map(pd.to_datetime) - df.PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365

    df["Momentum"] = df["S"] * df["PlayerWeight"]

    #df["F"] = df["A"] * df["PlayerWeight"]

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X"].agg(np.mean)[1::2])
    rusher_x = np.repeat(rusher_x, 22) # repeat each elemnt 22 times df["RusherX"]
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y"].agg(np.mean)[1::2])
    rusher_y = np.repeat(rusher_y, 22) # df["RusherY"]
    df["DisToRusher"] = np.sqrt((df["X"] - rusher_x) ** 2 + (df["Y"] - rusher_y) ** 2)
    df["TackleTimeToRusher"] = df["DisToRusher"] / df["S"] # includes nan when the speed of rusher is 0
    #df.loc[df.is_run==True, "TackleTimeToRusher"] = 0

    #df["Dir_sin"] = df["Dir"].apply(lambda x : np.sin((450-x) * np.pi/ 180))
    df["Dir_cos"] = df["Dir"].apply(lambda x : np.cos((450-x) * np.pi/ 180))
    #df["Orientation_sin"] = df["Orientation"].apply(lambda x : np.cos((450-x) * np.pi/ 180))
    #df["Orientation_cos"] = df["Orientation"].apply(lambda x : np.sin((450-x) * np.pi/ 180))
    
    df["Momentum_cos"] = df["Momentum"] * df["Dir_cos"]
    #df["Momentum_sin"] = df["Momentum"] * df["Dir_sin"]

    rusher_s = np.array(df.groupby(["PlayId", "is_run"]).agg(np.mean)["S"][1::2])
    #rusher_s[rusher_s == 0] = 1e-15 # replace velocity 0 with very small values
    rusher_s = np.repeat(rusher_s, 22)
    df["RatioSToRusher"] = df["S"] / rusher_s
    #df.loc[df.is_run==True, "RatioSToRusher"] = 1

    df_single = df[df.is_run==True].copy()
    
    #df_single["NecDisPerDown"] = df_single["Distance"] / (5 - df_single["Down"])
        
    df_single['time_quarter'] = df_single.GameClock.map(lambda x:transform_time_quarter(x))
    df_single['time_end'] = df_single.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)

    df_single['TimeHandoff'] = df_single['TimeHandoff'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df_single['TimeSnap'] = df_single['TimeSnap'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df_single['handoff_snap_diff'] = (df_single['TimeHandoff'] - df_single['TimeSnap']).map(lambda x:x.seconds)

    df_single["Stadium"] = df_single["Stadium"].map(lambda x: "Broncos Stadium at Mile High" if x=="Broncos Stadium At Mile High" 
                                             else ("CenturyLink Field" if x == "CenturyField" or x == x=="CenturyLink"
                                             else ("Everbank Field" if x == "EverBank Field"
                                             else ("FirstEnergy Stadium" if x =="First Energy Stadium" or x=="FirstEnergy" or x == "FirstEnergyStadium"
                                             else ("Lambeau Field" if x == "Lambeau field"
                                             else ("Los Angeles Memorial Coliseum" if x == "Los Angeles Memorial Coliesum"
                                             else ("M&T Bank Stadium" if x == "M & T Bank Stadium" or x == "M&T Stadium"
                                             else ("Mercedes-Benz Superdome" if x == "Mercedes-Benz Dome"
                                             else ("MetLife Stadium" if x == "MetLife" or x == "Metlife Stadium"
                                             else ("NRG Stadium" if x == "NRG"
                                             else ("Oakland-Alameda County Coliseum" if x == "Oakland Alameda-County Coliseum"
                                             else ("Paul Brown Stadium" if x == "Paul Brown Stdium"
                                             else ("Twickenham Stadium" if x == "Twickenham" else x)))))))))))))

    df_single["Location"] = df_single["Location"].map(lambda x: "Arlington, TX" if x == "Arlington, Texas"
                                            else ("Baltimore, MD" if x == "Baltimore, Maryland" or x == "Baltimore, Md."
                                            else ("Charlotte, NC" if x == "Charlotte, North Carolina"
                                            else ("Chicago, IL" if x == "Chicago. IL"
                                            else ("Cincinnati, OH" if x == "Cincinnati, Ohio"
                                            else ("Cleveland, OH" if x == "Cleveland" or x == "Cleveland Ohio" or x == "Cleveland, Ohio" or x == "Cleveland,Ohio"
                                            else ("Detroit, MI" if x == "Detroit"
                                            else ("East Rutherford, NJ" if x == "E. Rutherford, NJ" or x == "East Rutherford, N.J."
                                            else ("Foxborough, MA" if x == "Foxborough, Ma"
                                            else ("Houston, TX" if x == "Houston, Texas"
                                            else ("Jacksonville, FL" if x == "Jacksonville Florida" or x == "Jacksonville, Fl" or x == "Jacksonville, Florida"
                                            else ("London" if x == "London, England"
                                            else ("Los Angeles, CA" if x == "Los Angeles, Calif."
                                            else ("Miami Gardens, FLA" if x == "Miami Gardens, Fla."
                                            else ("New Orleans, LA" if x == "New Orleans" or x == "New Orleans, La."
                                            else ("Orchard Park, NY" if x == "Orchard Park NY"
                                            else ("Philadelphia, PA" if x == "Philadelphia, Pa."
                                            else ("Pittsburgh, PA" if x == "Pittsburgh"
                                            else ("Seattle, WA" if x == "Seattle" else x)))))))))))))))))))

    grass_labels = ['grass', 'natural grass', 'natural', 'naturall grass']
    df_single['Grass'] = np.where(df_single.Turf.str.lower().isin(grass_labels), "Natural", "Artificial")
                                                                 
    #top20_weather = list(df.GameWeather.value_counts(normalize=True, dropna=False).cumsum().head(20).index)
    #df_single["GameWeather"] = df_single["GameWeather"].apply(lambda x: "Others" if x not in top20_weather else x)
    
    #df_single['WindSpeed'] = df_single['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
    #df_single['WindSpeed'] = df_single['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    #df_single['WindSpeed'] = df_single['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    #df_single['WindSpeed'] = df_single['WindSpeed'].apply(str_to_float)
    
    #df_single['WindDirection'] = df_single['WindDirection'].apply(lambda x: "north" if x == "N" or x == "FROM S"
    #                                                   else ("south" if x == 'S' or x== 'FROM N'
    #                                                   else ("west" if x == 'W' or x == 'FROM E'
    #                                                   else ("east" if x == 'E' or x == 'FROM W'
    #                                                   else ("north east" if x == 'FROM SW' or x == 'FROM SSW' or x == 'FROM WSW'
    #                                                   else ("north west" if x == 'FROM SE' or x == 'FROM SSE' or x == 'FROM ESE'
    #                                                   else ("south east" if x == 'FROM NW' or x == 'FROM NNW' or x == 'FROM WNW'
    #                                                   else ("south west" if x == 'FROM NE' or x == 'FROM NNE' or x == 'FROM ENE'
    #                                                   else ("north west" if x == 'NW' or x == 'NORTHWEST'
    #                                                   else ("north east" if x == 'NE' or x == 'NORTH EAST'
    #                                                   else ("south west" if x == 'SW' or x == 'SOUTHWEST'
    #                                                   else ("south east" if x == 'SE' or x == 'SOUTHEAST' else "unknown"))))))))))))

    #outdoor =['Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 'Outdor', 'Ourdoor', 'Outside', 'Outddors', 
    #         'Outdoor Retr Roof-Open', 'Oudoor', 'Bowl']
    #indoor_closed = ['Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed',
    #                   'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed']
    #indoor_open = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
    #dome_closed = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
    #dome_open = ['Domed, Open', 'Domed, open']

    #df_single['StadiumType'] = df_single['StadiumType'].apply(lambda x: "outdoor" if x in outdoor 
    #                                                        else ("indoor closed" if x in indoor_closed
    #                                                        else ("indoor open" if x in indoor_open
    #                                                        else ("dome_closed" if x in dome_closed
    #                                                        else ("dome_open" if x in dome_open else "unknown")))))
    
    #rain = ['Rainy', 'Rain Chance 40%', 'Showers',
    #          'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
    #          'Scattered Showers', 'Cloudy, Rain', 'Rain shower', 'Light Rain', 'Rain']
    #overcast = ['Cloudy, light snow accumulating 1-3"', 'Party Cloudy', 'Cloudy, chance of rain',
    #              'Coudy', 'Cloudy, 50% change of rain', 'Rain likely, temps in low 40s.',
    #              'Cloudy and cold', 'Cloudy, fog started developing in 2nd quarter',
    #              'Partly Clouidy', '30% Chance of Rain', 'Mostly Coudy', 'Cloudy and Cool',
    #              'cloudy', 'Partly cloudy', 'Overcast', 'Hazy', 'Mostly cloudy', 'Mostly Cloudy',
    #              'Partly Cloudy', 'Cloudy']
    #clear = ['Partly clear', 'Sunny and clear', 'Sun & clouds', 'Clear and Sunny',
    #           'Sunny and cold', 'Sunny Skies', 'Clear and Cool', 'Clear and sunny',
    #           'Sunny, highs to upper 80s', 'Mostly Sunny Skies', 'Cold',
    #           'Clear and warm', 'Sunny and warm', 'Clear and cold', 'Mostly sunny',
    #           'T: 51; H: 55; W: NW 10 mph', 'Clear Skies', 'Clear skies', 'Partly sunny',
    #           'Fair', 'Partly Sunny', 'Mostly Sunny', 'Clear', 'Sunny']
    #snow = ['Heavy lake effect snow', 'Snow']
    #none = ['N/A Indoor', 'Indoors', 'Indoor', 'N/A (Indoors)', 'Controlled Climate']

    #df_single['GameWeather'] = train_df['GameWeather'].apply(lambda x: "rain" if x in rain 
    #                                                         else ("overcast" if x in overcast
    #                                                        else ("clear" if x in clear
    #                                                        else ("snow" if x in snow
    #                                                        else ("indoor" if x in none else "unknown")))))
                                                                 
    df_single["OffenseFormation"] = df_single["OffenseFormation"].fillna("Unknown") 
    df_single['DefendersInTheBox_vs_Distance'] = df_single['DefendersInTheBox'] / df_single['Distance']
                                                                 
    #df_single['back_oriented_down_field'] = df_single['Orientation'].apply(lambda x: back_direction(x))
    #df_single['back_moving_down_field'] = df_single['Dir'].apply(lambda x: back_direction(x))

    #arr = [[int(s[0]) for s in t.split(", ")] for t in df_single["DefensePersonnel"]]
    #df_single["DefenseDL"] = np.array([a[0] for a in arr])
    #df_single["DefenseLB"] = np.array([a[1] for a in arr])
    #df_single["DefenseDB"] = np.array([a[2] for a in arr])
    #df_single["DefenseOL"] = np.array([a[3] if len(a) == 4 else 0 for a in arr])
  
    #df_single["OffenseRB"] = df_single["OffensePersonnel"].apply(lambda x: 
    #                        int(x.replace(",", "").split(" RB")[0][-1]) if "RB" in x else 0)
    #df_single["OffenseTE"] = df_single["OffensePersonnel"].apply(lambda x: 
    #                        int(x.replace(",", "").split(" TE")[0][-1]) if "TE" in x else 0)
    #df_single["OffenseWR"] = df_single["OffensePersonnel"].apply(lambda x: 
    #                        int(x.replace(",", "").split(" WR")[0][-1]) if "WR" in x else 0)
    #df_single["OffenseOL"] = df_single["OffensePersonnel"].apply(lambda x: 
    #                        int(x.replace(",", "").split(" OL")[0][-1]) if "OL" in x else 0)
    #df_single["OffenseDL"] = df_single["OffensePersonnel"].apply(lambda x: 
    #                        int(x.replace(",", "").split(" DL")[0][-1]) if "DL" in x else 0)
    #df_single["OffenseQB"] = df_single["OffensePersonnel"].apply(lambda x: 
    #                        int(x.replace(",", "").split(" QB")[0][-1]) if "QB" in x else 0)
  
    #df_single["DisToQB"] = np.array(df[(df.Position=="QB") | (df.Position=="C")].groupby(["PlayId"]).agg(np.mean)["DisToRusher"])

    #df_single["OffenseFormation"] = df_single["OffenseFormation"].apply(lambda x: "SHOTGUN" if x== "ACE" else x)

    #df_single["Margin"] = df_single["HomeScoreBeforePlay"] - df_single["VisitorScoreBeforePlay"]
    #df_single.loc[df_single['Team'] == "away", 'Margin'] = (df_single['VisitorScoreBeforePlay'][df_single['Team'] == "away"] - df_single['HomeScoreBeforePlay'][df_single['Team'] == "away"])

    df_single['runner_height'] = df_single.PlayerHeight.map(transform_height)
    df_single['own_field'] = (df_single['FieldPosition'] == df_single['PossessionTeam']).astype(int)
    dist_to_end = df_single.apply(lambda x:(100 - x.loc['YardLine']) if x.loc['own_field']==1 else x.loc['YardLine'],axis=1)
    df_single.drop(remove_features,axis=1,inplace=True) 

    tmp = df.groupby(["PlayId", "OnOffense"]).agg(np.mean)[["X", "Y"]]
    df_single["DefenseAveX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseAveX"] = np.array(tmp[1::2]["X"])

    df_single["DefenseAveY"] = np.array(tmp[0::2]["Y"]) 
    df_single["OffenseAveY"] = np.array(tmp[1::2]["Y"]) 
    
    #df_single["DefenseAveAge"] = np.array(tmp[0::2]["age"])
    #df_single["OffenseAveAge"] = np.array(tmp[1::2]["age"])

    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X", "Y"]]
    df_single["DefenseStdX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseStdX"] = np.array(tmp[1::2]["X"])

    df_single["DefenseStdY"] = np.array(tmp[0::2]["Y"])
    df_single["OffenseStdY"] = np.array(tmp[1::2]["Y"])

    df_single["RunnerToDefenseCentoid"] = np.sqrt((df_single["X"] - df_single["DefenseAveX"]) ** 2 + (df_single["Y"] - df_single["DefenseAveY"]) ** 2)
    df_single["RunnerToOffenseCentoid"] = np.sqrt((df_single["X"] - df_single["OffenseAveX"]) ** 2 + (df_single["Y"] - df_single["OffenseAveY"]) ** 2)

    tmp_max = df.groupby(["PlayId", "OnOffense"])["X"].max()
    tmp_min = df.groupby(["PlayId", "OnOffense"])["X"].min()
    df_single["DefenseSpreadX"] = np.array(tmp_max[0::2]- tmp_min[0::2])
    df_single["OffenseSpreadX"] = np.array(tmp_max[1::2]- tmp_min[1::2])

    df_single["RunnerToScrimmage"] = df_single["X"] - df_single["YardLine"]

    df_single["MinTackleTime"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher"].min()[0::2])
    #df_single["1stDefender_Momentum_cos"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["Momentum_cos"])
    #df_single["1stDefender_Momentum_sin"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["Momentum_sin"])
    #df_single["1stDefender_A"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["A"])

    #df_single["Rusher1stDefSpeedRatio"] = df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["RatioSToRusher"]

    pts = np.array(df[["X", "Y"]]).reshape(df.shape[0]//22, 22, 2) # plays * players * (X, Y, rusher)
    rusher_index = list(df[df.is_run==True].index % 22) 
    closest_def_index = list(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]].index % 22)
    rusher_voronoi = []
    closest_def_voronoi = []

    for i in range(0, df.shape[0] //22):
        rusher_voronoi.append(voronoi_volumes(pts[i], rusher_index[i]))
        closest_def_voronoi.append(voronoi_volumes(pts[i], closest_def_index[i]))
    df_single["RusherVoronoi"] = rusher_voronoi    
    df_single["FirstDefenderVoronoi"] = closest_def_voronoi 
    df_single.fillna(-999,inplace=True) 
    #remove_features2 = ["OnOffense", "DisToRusher"], "TackleTimeToRusher", "RatioSToRusher"]
    #df_single.drop(remove_features2, axis=1, inplace=True)
    
    if "Yards" not in df_single.columns:
        df_single.drop(["GameId", 'PossessionTeam'], axis=1, inplace=True)

    return df_single, dist_to_end

In [7]:
train_single, dist_to_end_train = transform_data(train)
y_train = train_single.Yards
X_train = train_single.drop(['Yards'],axis=1)
for f in X_train.columns:
    if X_train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f])+[-999])
        X_train[f] = lbl.transform(list(X_train[f]))
X_train_first = X_train.drop(['GameId', 'PossessionTeam'],axis=1)
cdf = get_cdf_df(y_train).values.reshape(-1,)

# modelling

In [8]:
# validation 1
n_folds=5
kf=KFold(n_splits = n_folds, random_state=1108)
resu1 = 0
resu2_cprs = 0
resu3_mae=0
stack_train = np.zeros([X_train.shape[0],])
models = []
lgbm_params = {
    "objective" : "regression",
    "metric" : "mae", 
    "tree_learner": "serial",
    "max_depth" : -1,
    "boosting": 'gbdt',
    #"num_leaves" : 13,
    "learning_rate" : 0.1,
    #"bagging_freq": 5,
    #"bagging_fraction" : 0.4,
    #"feature_fraction" : 0.05,
    #"min_data_in_leaf": 80,
}
feature_importance_df = pd.DataFrame(list(X_train_first.columns), columns=["Feature"])
for i , (train_index, test_index) in enumerate(kf.split(X_train_first, y_train)):
    X_train2= X_train_first.iloc[train_index,:]
    y_train2= y_train.iloc[train_index]
    X_test2= X_train_first.iloc[test_index,:]
    y_test2= y_train.iloc[test_index]
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
    
    clf = lgb.train(
        lgbm_params, lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=100000,
        early_stopping_rounds=100,
    )
    
    models.append(clf)
    temp_predict = clf.predict(X_test2)
    stack_train[test_index] = temp_predict
    mse = mean_squared_error(y_test2, temp_predict)
    crps = CRPS_pingyi1(temp_predict,y_test2,4,cdf,dist_to_end_train.iloc[test_index])
    mae = mean_absolute_error(y_test2, temp_predict)
    print(crps)
    #print(crps_score(temp_predict,y_test2))
    
    resu1 += mse/n_folds
    resu2_cprs += crps/n_folds
    resu3_mae += mae/n_folds
    feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
    gc.collect()
print('mean mse:',resu1)
print('oof mse:',mean_squared_error(y_train,stack_train))
print('mean mae:',resu3_mae)
print('oof mae:',mean_absolute_error(y_train,stack_train))
print('mean cprs:',resu2_cprs)
print('oof cprs:',CRPS_pingyi1(stack_train,y_train,4,cdf,dist_to_end_train))
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

[1]	valid_0's l1: 3.78148
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 3.71156
[3]	valid_0's l1: 3.66778
[4]	valid_0's l1: 3.61949
[5]	valid_0's l1: 3.58954
[6]	valid_0's l1: 3.55798
[7]	valid_0's l1: 3.53686
[8]	valid_0's l1: 3.5046
[9]	valid_0's l1: 3.49249
[10]	valid_0's l1: 3.47322
[11]	valid_0's l1: 3.46322
[12]	valid_0's l1: 3.44233
[13]	valid_0's l1: 3.43283
[14]	valid_0's l1: 3.42673
[15]	valid_0's l1: 3.42046
[16]	valid_0's l1: 3.40599
[17]	valid_0's l1: 3.3993
[18]	valid_0's l1: 3.39573
[19]	valid_0's l1: 3.38556
[20]	valid_0's l1: 3.38305
[21]	valid_0's l1: 3.37774
[22]	valid_0's l1: 3.37616
[23]	valid_0's l1: 3.37354
[24]	valid_0's l1: 3.36849
[25]	valid_0's l1: 3.35857
[26]	valid_0's l1: 3.35757
[27]	valid_0's l1: 3.3566
[28]	valid_0's l1: 3.3562
[29]	valid_0's l1: 3.35795
[30]	valid_0's l1: 3.36093
[31]	valid_0's l1: 3.35562
[32]	valid_0's l1: 3.35187
[33]	valid_0's l1: 3.35352
[34]	valid_0's l1: 3.35093
[35]	valid_0's l1: 3.35554
[36]	v

In [9]:
X_train_first.columns

Index(['Team', 'S', 'A', 'Dis', 'NflId', 'JerseyNumber', 'Season', 'Quarter',
       'Down', 'Distance', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'OffenseFormation', 'OffensePersonnel', 'DefendersInTheBox',
       'DefensePersonnel', 'PlayerWeight', 'PlayerCollegeName', 'Position',
       'Week', 'Stadium', 'Location', 'StadiumType', 'GameWeather',
       'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'OnOffense',
       'YardLine', 'X', 'Y', 'Orientation', 'Dir', 'age', 'Momentum',
       'DisToRusher', 'TackleTimeToRusher', 'Dir_cos', 'Momentum_cos',
       'RatioSToRusher', 'time_quarter', 'time_end', 'handoff_snap_diff',
       'Grass', 'DefendersInTheBox_vs_Distance', 'runner_height',
       'DefenseAveX', 'OffenseAveX', 'DefenseAveY', 'OffenseAveY',
       'DefenseStdX', 'OffenseStdX', 'DefenseStdY', 'OffenseStdY',
       'RunnerToDefenseCentoid', 'RunnerToOffenseCentoid', 'DefenseSpreadX',
       'OffenseSpreadX', 'RunnerToScrimmage', 'MinTackleTime', 'R

In [10]:
feature_importance_df.sort_values("Average").tail(10)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
56,RunnerToOffenseCentoid,44,34,26,8,4,23.2,15.210523,0.655626
6,Season,28,28,24,20,17,23.4,4.363485,0.186474
38,Dir_cos,33,33,32,17,15,26.0,8.197561,0.315291
1,S,41,41,38,15,18,30.6,11.603448,0.379198
59,RunnerToScrimmage,57,53,35,7,7,31.8,21.562931,0.678080
57,DefenseSpreadX,38,42,46,22,12,32.0,12.899612,0.403113
51,DefenseStdX,59,51,49,22,14,39.0,17.652195,0.452620
60,MinTackleTime,76,62,54,18,10,44.0,25.612497,0.582102
39,Momentum_cos,68,64,58,22,17,45.8,21.766029,0.475241
2,A,99,87,78,47,38,69.8,23.438430,0.335794


In [11]:
# validation 2 from https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/113861#latest-658998
def NFL_validation_split(df):
    games = df[['GameId', 'PossessionTeam']].drop_duplicates()

    # Sort so the latest games are first and label the games with cumulative counter
    games = games.sort_values(['PossessionTeam', 'GameId'], ascending=[True, False])
    games['row_number'] = games.groupby(['PossessionTeam']).cumcount() + 1

    # Use last 5 games for each team as validation. There will be overlap since two teams will have the same
    # GameId
    game_set = set([1, 2, 3, 4, 5])

    # Set of unique game ids
    game_ids = set(games[games['row_number'].isin(game_set)]['GameId'].unique().tolist())

    return game_ids

game_ids = NFL_validation_split(X_train)
X_train_second = X_train[~X_train['GameId'].isin(game_ids)].copy()
X_test_second = X_train[X_train['GameId'].isin(game_ids)].copy()
X_train_second.drop(["GameId", 'PossessionTeam'],axis=1, inplace=True)
X_test_second.drop(["GameId", 'PossessionTeam'],axis=1, inplace=True)
train_inds, test_inds = X_train_second.index, X_test_second.index
y_train_second, y_test_second = y_train[train_inds], y_train[test_inds]

lgbm_params = {
    "objective" : "regression",
    "metric" : "mae", 
    "tree_learner": "serial",
    "max_depth" : -1,
    "boosting": 'gbdt',
    #"num_leaves" : 13,
    "learning_rate" : 0.1,
    #"bagging_freq": 5,
    #"bagging_fraction" : 0.4,
    #"feature_fraction" : 0.05,
    #"min_data_in_leaf": 80,
}
feature_importance_df = pd.DataFrame(list(X_train_second.columns), columns=["Feature"])
lgb_train = lgb.Dataset(X_train_second, y_train_second)
lgb_eval = lgb.Dataset(X_test_second, y_test_second, reference=lgb_train)
clf = lgb.train(
        lgbm_params, lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=100000,
        early_stopping_rounds=100,
    )
temp_predict = clf.predict(X_test2)
feature_importance_df["Importance"] = clf.feature_importance()
stack_train[test_index] = temp_predict
mse = mean_squared_error(y_test2, temp_predict)
crps = CRPS_pingyi1(temp_predict,y_test2,4,cdf,dist_to_end_train.iloc[test_index])
mae = mean_absolute_error(y_test2, temp_predict)
print(crps)

[1]	valid_0's l1: 3.92733
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 3.91904
[3]	valid_0's l1: 3.92
[4]	valid_0's l1: 3.91672
[5]	valid_0's l1: 3.90693
[6]	valid_0's l1: 3.89935
[7]	valid_0's l1: 3.89671
[8]	valid_0's l1: 3.89991
[9]	valid_0's l1: 3.89619
[10]	valid_0's l1: 3.89935
[11]	valid_0's l1: 3.89581
[12]	valid_0's l1: 3.89865
[13]	valid_0's l1: 3.89717
[14]	valid_0's l1: 3.89169
[15]	valid_0's l1: 3.89904
[16]	valid_0's l1: 3.90305
[17]	valid_0's l1: 3.9003
[18]	valid_0's l1: 3.89805
[19]	valid_0's l1: 3.90364
[20]	valid_0's l1: 3.90379
[21]	valid_0's l1: 3.9077
[22]	valid_0's l1: 3.90871
[23]	valid_0's l1: 3.91265
[24]	valid_0's l1: 3.91397
[25]	valid_0's l1: 3.91272
[26]	valid_0's l1: 3.91538
[27]	valid_0's l1: 3.91777
[28]	valid_0's l1: 3.91672
[29]	valid_0's l1: 3.91432
[30]	valid_0's l1: 3.9143
[31]	valid_0's l1: 3.91717
[32]	valid_0's l1: 3.91905
[33]	valid_0's l1: 3.92375
[34]	valid_0's l1: 3.92596
[35]	valid_0's l1: 3.92784
[36]	val

In [12]:
feature_importance_df.sort_values("Importance").tail(10)

,Feature,Importance
57,DefenseSpreadX,13
59,RunnerToScrimmage,14
14,DefendersInTheBox,18
39,Momentum_cos,22
38,Dir_cos,22
60,MinTackleTime,22
6,Season,23
51,DefenseStdX,23
1,S,26
2,A,56


# prediction

In [13]:
for (test_df, sample_prediction_df) in env.iter_test():
    X_test, dist_to_end_test = transform_data(test_df)
    if set(X_test.columns) != set(X_train_first.columns):
        print("columns are different")
        break
    for f in X_test.columns:
        if X_test[f].dtype=='object':
            X_test[f] = X_test[f].map(lambda x:x if x in set(X_train_first[f]) else -999)
    for f in X_test.columns:
        if X_test[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train_first[f])+[-999])
            X_test[f] = lbl.transform(list(X_test[f])) 
    pred_value = 0
    for model in models:
        pred_value += model.predict(X_test)[0]/5
    pred_data = list(get_score(pred_value,cdf,4,dist_to_end_test.values[0]))
    pred_data = np.array(pred_data).reshape(1,199)
    pred_target = pd.DataFrame(index = sample_prediction_df.index, \
                               columns = sample_prediction_df.columns, \
                               #data = np.array(pred_data))
                               data = pred_data)
    env.predict(pred_target)
env.write_submission_file()

Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.
